In [1]:
import pandas as pd
import numpy as np

import ast
import glob
import json
import operator
import os
import path
import re

In [2]:
os.getcwd()

'/Users/fordhiggins/msan_files/msan698/findyourpath/code'

In [3]:
! ls ../Data/Resume_cleaned/ordered_profiles

../Data/Resume_cleaned/ordered_profiles


In [4]:
resumes = []
with open('../Data/Resume_cleaned/ordered_profiles') as f:
  for line in f:
      line = line.strip()
      line = ast.literal_eval(line)
      resumes.append(line)

In [5]:
df_resumes = pd.DataFrame(resumes)
df_resumes.head(3)

,Job_Title,events,userID
0,architect,"[(2010, bachelor of science in architecture, 0...",architect_0
1,architect,"[(2001, high school, 0), (2003, higher seconda...",architect_1
2,architect,"[(2016, junior pega architect, 1), (2020, bach...",architect_2


In [6]:
df_resumes.Job_Title.unique()

array(['architect', 'auditor', 'business_analyst', 'business_strategist',
       'data_analyst', 'data_engineer', 'data_scientist',
       'Engineering Manager.csv', 'Finance Manager.csv',
       'Financial Analyst.csv', 'Human Resource Manager.csv',
       'Marketing Director.csv', 'Operation Analyst.csv',
       'Operation Manager.csv', 'Product Manager.csv', 'Recruiter.csv',
       'Sales Associate.csv', 'Sales Manager.csv',
       'Software Architect.csv', 'Software Engineer.csv',
       'VP of Engineering.csv', 'VP of Marketing.csv',
       'VP of Operations.csv'], dtype=object)

In [7]:
titles = pd.Series(df_resumes.Job_Title.unique())
titles.str.split('.', expand=True).iloc[:, 0].str.replace(' ', '_').str.lower()

0                  architect
1                    auditor
2           business_analyst
3        business_strategist
4               data_analyst
5              data_engineer
6             data_scientist
7        engineering_manager
8            finance_manager
9          financial_analyst
10    human_resource_manager
11        marketing_director
12         operation_analyst
13         operation_manager
14           product_manager
15                 recruiter
16           sales_associate
17             sales_manager
18        software_architect
19         software_engineer
20         vp_of_engineering
21           vp_of_marketing
22          vp_of_operations
Name: 0, dtype: object

In [8]:
len(titles)

23

**Groupings:**
1. Business:
    - business_analyst
    - business_strategist
    - operation_analyst
    - operation_manager
    - product_manager
    - vp_of_operations
2. Finance:
    - finance_manager
    - financial_manager
3. Data/Software:
    - data_analyst
    - data_engineer
    - data_scientist
    - engineering_manager (?)
    - software_architect
    - software_engineer
    - vp_of_engineering
4. Marketing/Sales:
    - marketing_director
    - recruiter
    - sales_manager
    - sales_associate
    - vp_of_marketing
5. Unsure:
    - architect
    - auditor
    - human_resource_manager

In [9]:
my_titles = ['Sales_Manager', 'Software_Architect', 'Software_Engineer', 'VP_of_Engineering', 'VP_of_Marketing']
my_titles = [x.lower() for x in my_titles]

df_resumes.Job_Title = df_resumes.Job_Title.str.split('.', expand=True).iloc[:, 0].str.replace(' ', '_').str.lower()

In [10]:
for i in np.arange(len(my_titles)):
    print df_resumes[df_resumes.Job_Title == my_titles[i]].shape

(30, 3)
(259, 3)
(385, 3)
(247, 3)
(183, 3)


In [11]:
def print_events(num):
    df = df_resumes[df_resumes.Job_Title == my_titles[num]].reset_index(drop = True)
    for i in np.arange(len(df)):
        print i, df.events[i]
        print
    

In [12]:
sm_to_keep = [1, 3, 4, 7, 9, 10, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 27, 28, 29]
sa_to_keep = [1, 5, 7, 8, 9, 12, 15, 21, 22, 29, 36, 68, 82, 91, 106, 129, 155, 221, 228, 241]
se_to_keep = [1, 2, 8, 15, 22, 44, 56, 85, 94, 100, 102, 108, 128, 157, 165, 203, 226, 247, 272, 278]
vpe_to_keep = [3, 15, 18, 43, 44, 55, 57, 71, 72, 89, 111, 127, 128, 133, 147, 165, 166, 189, 233, 238]
vpm_to_keep = [5, 21, 38, 44, 48, 66, 72, 73, 77, 82, 83, 87, 101, 104, 110, 119, 128, 129, 130, 154]
to_keep = [sm_to_keep, sa_to_keep, se_to_keep, vpe_to_keep, vpm_to_keep]

In [13]:
df_keep = pd.DataFrame()
for i in np.arange(len(my_titles)):
    df = df_resumes[df_resumes.Job_Title == my_titles[i]].reset_index(drop = True)
    df_keep = pd.concat([df_keep, df.iloc[to_keep[i]]])

df_keep = df_keep.reset_index(drop = True)

In [14]:
title_map_ford = {
    'bpm architect': 'software architect'
    , 'enterprise architect': 'software architect'
    , 'solutions architect': 'software architect'
    , 'technology architect': 'software architect'
    , 'sharepoint developer / architect': 'software architect'
    , 'services architect': 'software architect'
    , 'sr software engineer (consultant)': 'senior software engineer'
    , 'senior architect/team lead': 'senior architect'
    , 'chief architect': 'senior architect' 
    , 'technical software system architect': 'software architect'
    , 'bi architect/developer': 'software architect'
    , 'vp of operations - engineering': 'vp of engineering'
    , 'technical architect / software development lead': 'software architect'
    , 'software engineering & it services': 'software engineer' 
    , 'solution architect': 'software architect'
    , 'mba in marketing': 'master in business administration'
    , 'mba': 'master in business administration'
    , 'ms in computer science': 'master of science in computer science'
    , 'bs in computer science': 'bachelor of science in computer science'
    , 'senior development engineer': 'senior engineer' 
    , 'vp of marketing and sales': 'vp of marketing'
    , 'sr software engineer / architect': 'senior software engineer' 
    , 'software engineer (director level)': 'senior software engineer'
    , 'lead architect': 'senior software architect'
    , 'information architect': 'data architect'
    , 'technical director of software & development': 'vp of software' #???
    , 'director and vp of sales and marketing': 'vp of marketing'
    , 'master of business administration in business administration': 'master in business administration'
    , 'diploma in business': 'bachelor in business administration'
    , 'ms in information technology and management': 'master of science in information technology'
    , 'sr solutions architect': 'senior architect'
    , 'mba in finance': 'master in business administration'
    , 'bs in business adminstration & accontancy': 'bachelor of science in business administration'
    , 'sql data architect': 'data architect'
    , 'principal architect': 'software architect'
    , 'sr project engineer gas engines': 'senior engineer'
    , 'phd in comptation chemistry': 'phd in computational chemistry'
    , 'vp of devops/cloud engineering and it': 'vp of engineering'
    , 'cto and chief architect': 'chief technology officer' 
    , 'bachelor of science in electronics and commnications engineering': 'bachelor of science in electrical engineering'
    , 'bachelor of computer science in computer science': 'bachelor of science in computer science'
    , 'msc in information systems in information systems': 'master of science in information technology'
    , 'ms in civil engineering in education': 'master of science in civil engineering'
    , 'bachelor of science in computer engineering in computer engineering': 'bachelor of science in computer science' # sci instead of eng?
    , 'software system architect': 'software architect'
    , 'vp engineering services': 'vp of engineering'
    , 'director of sales': 'vp of sales' 
    , 'data base architect': 'data architect'
    , 'chief engineer': 'senior engineer'
    , 'bsc in computer science smma': 'bachelor of science in computer science'
    , 'masters in computer science': 'master of science in computer science'
    , 'architect/team lead': 'software architect'
    , 'be in electrical & electronics': 'bachelor of science in electrical engineering' 
    , 'ms in computer science in computer science': 'master of science in computer science'
    , 'data warehouse architecture design': 'data architect'
    , 'bachelor of business administration in finance & marketing': 'bachelor in business administration'
    , 'associate software engineer': 'software engineer' # ???
    , 'ba in english': 'bachelor of arts in english'
    , 'ms in chemical engineering': 'master of science in chemical engineering'
    , 'bs in physics': 'bachelor of science in physics'
    , 'bachelor of science in compting and information system in compting and information system': 'bachelor of science in information technology' 
    , 'master gradate in leadership and relationship bilding': 'master graduate in leadership and relationship building'
    , 'masters of science degree in computer science': 'master of science in computer science'
    , 'senior software database architect/engineer': 'senior software engineer' # ???
    , 'director of engineering and operations': 'vp of engineering' # vp instead of director?
    , 'corporate director of marketing': 'vp of marketing'
    , 'be in electronics': 'bachelor of science in electrical engineering'
    , 'pd in computer science & systems management': 'phd in computer science'
    , 'bs in mechanical engineering': 'bachelor of science in mechanical engineering'
    , 'bachelor of business administration in management information systems': 'bachelor in business administration'
    , 'global development center site architect': 'architect'
    , 'ms in information systems': 'master of science in information technology'
    , 'unit systems security architect': 'data architect'
    , 'bachelors degree in computer science in computer science': 'bachelor of science in computer science'
    , 'bachelor of arts in business management leadership in resorces & hospitality management': 'bachelor in business administration'
    , 'mba in technology management': 'master of business administration'
    , 'sr director of software engineering': 'senior software engineer'
    , 'ms in computer science in data mining and pattern recognition': 'master of science in computer science' # ???
    , 'etl data architect': 'data architect'
    , 'in chemical engineering': 'bachelor of science in chemical engineering'
    , 'be in electronics and telecommunication': 'bachelor of science in electrical engineering' # ???
    , 'director of marketing & inside sales': 'vp of marketing' # vp instead of director
    , 'ms in computer engineering': 'master of science in computer engineering'
    , 'ms in information systems in (msis)': 'master of science in information technology'
    , 'bs in software engineering in software engineering': 'bachelor of science in software engineering'
    , 'vice president of engineering': 'vp of engineering'
    , 'master in software engineering': 'master of science in software engineering' #??
    , 'director of sales and marketing': 'vp of marketing'
    , 'director of infrastructure and cloud systems': 'senior data architect' 
    , 'bachelor in jornalism': 'bachelor of arts in journalism'
    , 'director engineering': 'vp of engineering' # vp instead of director
    , 'sr business intelligent architect': 'senior architect'
    , 'senior vp of engineering': 'vp of engineering'
    , 'bs in engineering': 'bachelor of science in engineering'
    , 'bachelors of science in mechanical engineering': 'bachelor of science in mechanical engineering'
    , 'sr software engineer/ architect/sme': 'senior software engineer' # ???
    , 'med in edcational technology': 'master of education in educational technology'
    , 'senior cloud engineer': 'senior software engineer' # software or data?
    , 'ba in sport management': 'bachelor of arts in sport management'
    , 'manufacturing engineering': 'bachelor of science in engineering' 
    , 'director of advanced manufacturing engineering': 'vp of engineering' 
    , 'solution/application architect': 'software architect'
    , 'masters of business administration in business administration': 'master in business administration'
    , 'sales manager/sales analyst': 'sales manager'
    , 'sr director of production operations': 'vp of operations'
    , "lead data architect; csc (dhs'": 'data architect'
    , 'vp of operations/plant manager': 'vp of operations'
    , 'security architect': 'software architect'
    , 'spectrum engineering consortium ltd - dhaka': 'software engineer'
    , 'mba in fll-time program': 'master in business administration'
    , 'bs in electrical engineering': 'bachelor of science in electrical engineering'
    , 'be in network engineering in network engineering': 'bachelor of engineering in network engineering'
    , 'bachelor of arts in natral science': 'bachelor of arts in natural science'
    , 'data warehouse programmer analyst': 'data architect' # ???
    , 'software architect / consultant': 'software architect'
    , 'bachelor of science in ba in marketing': 'bachelor of arts in marketing'
    , 'bachelor of arts in commnications': 'bachelor of arts in communications'
    , 'bachelors in psychology': 'bachelor of science in psychology'
    , 'ba in marketing': 'bachelor of arts in marketing'
    , 'bs in commnity development': 'bachelor of science in community development'
    , 'bs in business ed': 'bachelor of science in business ed'
    , 'engineering manager & architect': 'data architect'
    , 'system grop': 'system group'
    , 'senior vp of operations': 'vp of operations'
    , 'executive mba in business': 'master in business administration' 
    , 'lead software engineer': 'software engineer'
    , 'ms in mechanical engineering': 'master of science in mechanical engineering'
    , 'bs in computer engineering': 'bachelor of science in computer engineering' # same as comp sci?
    , 'solution architect/administrator': 'software architect'
    , 'bachelor of electronic and information engineering in electronic and information engineering': 'bachelor of science in electrical engineering and information technology' # just change to ee?
    , 'bs in computer science and engineering': 'bachelor of science in computer science'
    , 'vice presient engineering & operations': 'vp of engineering'
    , 'sr solution architect': 'senior software architect'
    , 'bs in commnications': 'bachelor of science in communications'
    , 'ms in electrical engineering in computer science': 'master of science in computer science'#???
    , 'software architect/tech lead': 'software architect'
    , 'product engineer': 'software engineer' # ??
    , 'sr engineer / mechanical design supervisor': 'senior software engineer'
    , 'bs in chemical engineering': 'bachelor of science in chemical engineering'
    , 'senior cloud and mobile software engineer': 'senior software engineer'
    , 'data warehouse architect': 'data architect'
    , 'instrumentation & controls engineer': 'software engineer'
    , 'master of electrical engineering in electrical engineering': 'master of science in electrical engineering'
    , 'ms in cybersecrity and information assrance': 'master of science in cybersecurity'
    , 'bi/dw solution architect and delivery manager': 'software architect'
    , 'corses in computer science': 'courses in computer science'
    , 'ms in computer science in science and engineering': 'master of science in computer science' # ??
    , 'engineering manager/ architect': 'senior software architect' # ???
    , 'senior java software engineer | project lead' : 'senior software engineer'
    , 'bachelors candidate in computer science': 'bachelor of science in computer science'
    , 'ms in applied mathematics in applied mathematics': 'master of science in applied mathematics'
    , 'consulting database architect': 'data architect'
    , 'bachelors in economics': 'bachelor of science in economics'
    , 'bachelor of engineering in electronics and telecommnications': 'bachelor of science in electrical engineering'
    , 'systems integration architect': 'software architect'
    , 'azure bi architect/developer': 'software architect'
    , 'bsn in business commnication': 'bachelor of science in business communication'
    , 'senior software analyst - technical leader': 'senior software analyst' 
    , 'ba in computer science': 'bachelor of science in computer science'
    , 'senior software engineer/team lead': 'senior software engineer'
    , 'bachelor of technology in electrical and electronics engineering': 'bachelor of science in electrical engineering'
    , 'bs in chemical and biological engineering': 'bachelor of science in chemical and biological engineering'
    , 'director product management & engineering': 'vp of engineering'
    , 'engineering & supply chain': 'data engineer' # ?
    , 'ms in information management': 'master of science in information technology'
    , 'sales manager/ engineer': 'engineer'
    , 'ba in chemistry': 'bachelor of arts in chemistry'
    , 'ms in engineering and information systems': 'master of science in engineering and information technology'
    , 'bachelors in business marketing': 'bachelor of arts in business marketing'
    , 'master of science in computer science in distance education': 'master of science in computer science'
    , 'bs in advertising & psychology': 'bachelor of science in advertising and psychology'
    , 'vp engineering & product development': 'vp of engineering'
    , 'mba in business': 'master in business administration'
    , 'advisory software innovation engineer': 'software engineer'
    , 'software engineer - intern': 'software engineer intern'
    , 'architecture and sensitive information storage': 'data architect'
    , 'vp of engineering / vp of architecture': 'vp of engineering'
    , 'engineering manager / software architect': 'software architect'
    , 'systems engineer/lead data management': 'data engineer'
    , 'masters of science in logistics management': 'master of science in logistics management'
    , 'system engineer': 'data engineer'
    , 'sharepoint admin / architect': 'software architect'
    , 'msc in computer science': 'master of science in computer science'
    , 'sr sql data architect': 'senior data architect'
    , 'chief architect/dev manager': 'senior software architect'
    , 'sr engineering program manager': 'senior software engineer'
    , 'systems/enterprise architect': 'software architect'
    , 'ecm architect': 'data architect'
    , 'database architect/dba manager': 'data architect'
    , 'be in electronics and commnications engineering': 'bachelor of science in electrical and communications engineering'
    , 'associate in science in business administration in business administration': 'associate in business administration'
    , 'staff software engineer': 'software engineer'
    , 'fll stack software development': 'software engineer' 
    , 'business systems analyst': 'systems analyst'
    , 'bi/dw solution architect': 'software architect'
    , 'bi developer/architect': 'software architect'
    , 'security engineer': 'software engineer'
    , 'bs in political science': 'bachelor of science in political science'
    , 'sr software engineer/ architect': 'senior software engineer'# ???
    , 'software engineering team lead': 'senior software engineer'
    , 'ssrs architect/developer': 'software architect'
    , 'tech lead/ tech architect': 'data architect'
    , 'ms in electrical engineering': 'master of science in electrical engineering'
    , 'bs in information and computer science': 'bachelor of science in information technology and computer science'
    , 'technical architect': 'data architect'
    , 'bs in business administration': 'bachelor in business administration'
    , 'mba in business administration': 'master in business administration'
    , 'vp software engineering': 'vp of software engineering'
    , 'ms in management information systems': 'master of science in information technology'
    , 'ms in engineering in engineering': 'master of science in engineering'
    , 'bs in computer information systems': 'bachelor of science in information technology'
    , 'vp engineering & operations': 'vp of engineering'
    , 'ms in medical sciences': 'master of science in medical sciences'
    , 'as in physics and chemistry': 'associates in physics and chemistry'
    , 'bachelor of technology in computers': 'bachelor of science in information technology'
    , 'ms in logistics management': 'master of science in logistics management'
    , 'software architect (director level)': 'senior software architect'
    , 'bs in aerospace engineering': 'bachelor of science in aerospace engineering'
    , 'bs in marketing': 'bachelor of science in marketing'
    , 'ms degree in computer science': 'master of science in computer science'
    , 'bs': 'bachelor of science'
    , 'bs in ba in marketing': 'bachelor of arts in marketing'
    , 'bs in marketing and business administration': 'bachelor in business administration'
    , 'bs in computer engineering in computer engineering': 'bachelor of science in computer engineering'
    , 'principal systems analyst': 'senior systems analyst'
    , 'master of arts in marketing commnication': 'master of arts in marketing'
    , 'ba in business administration': 'bachelor in business administration'
    , 'bachelor of engineering in engineering': 'bachelor of science in engineering'
    , 'crrent in radiology/psychology': 'current in radiology/psychology'
    , 'senior software development architect/engineer': 'senior software engineer'
    , 'certificate of advanced stdies in information secrity management': 'certificate of advanced studies in information technology'
    , 'exective mba in business': 'master in business administration'
    , 'ms in computer science in distance edcation': 'master of science in computer science'
    , 'bs in business administration & accontancy': 'bachelor in business administration'
    , 'bs in electronics and commnications engineering': 'bachelor of science in electrical engineering'
    , 'bs in compting and information system in compting and information system': 'bachelor of science in information technology'
    , 'ms in software engineering': 'master of science in software engineering'
}

title_map = {
    u'amin and shah associates - ahmedabad': '',
 u'software engineer & architect' : 'software engineer',
 u'data science' : 'data science',
 u'u"financial analyst' : 'financial analyst',
 u'u"vp of marketing & business development': 'vp marketing',
 u'engineering': 'engineer',
 u'engineering manager': 'engineer manager',
 u'in advertising' : 'advertising',
 u'sales associate' : 'sales',
 u'lead technical / business strategist / dmts' : 'business analyst',
 u'strategy director / dmts' : 'business analyst',
 u'product architects' : 'product architect',
 u'PharmD' : 'doctor',
 u'microprocessor engineer - team lead' : 'engineer',
 u'B.S. in Electrical & Electronics' : 'Bachelor of Science in Electrical & Electronics',
 u'recruiter' : 'recruiter',
 u'director of facilities & plant operations' : 'director of operations',
 u'M.S. in Statistics' : 'Master of Science in Data science',
 u'B.S. in Mechanical Engineering in With a focs in Mechatronics' : 'Bachelor of Science in Mechanical Engineering',
 u'junior architect' : 'architect',
 u'staff it auditor' : 'auditor',
 u'bachelor of arts in economics' : 'bachelor of arts',
 u'design and marketing director' : 'marketing director',
 u'it auditor': 'auditor',
 u'"Bachelors in Healthcare Management/Administration"' : 'Bachelors in Administration',
 u'thousands of data analysts' : 'data science',
 u'business analyst': 'business analyst',
 u'and backend engineers' : 'software engineer',
 u'u"recruiter' : 'recuiter',
 u'bachelor of science in medical technology' : 'bachelor of science in medicine',
 u'associate' : 'associate',
 u'vp of operations' : 'vp of operations',
 u'degree_2 :' : '',
 u'diploma' : '',
 u'software engineer' : 'software engineer',
 u'u"sales associate' : 'sales',
 u'bachelor of science in mechanical engineering' : 'bachelor of science in mechanical engineering',
 u'high school or eqivalent' : '',
 u'managed and developed junior analyst' : 'data science',
 u'supervise recruiter' : 'recruiter',
 u'degree_1 :' : '',
 u'engineers' : 'engineer',
 u'b.s in fashion design' : 'bachelor of science in fashion',
 u'product and engineering teams' : 'engineer',
 u'u"senior systems engineer' : 'software engineer',
 u'senior revenue analyst' : 'business analyst',
 u'lead technical / business strategist/ dmts':'business analyst',
 u'mechanical design engineer':'engineer',
 u'"bachelors"' : 'bachelor of science in mechanical engineering',
 u'b.s. in electrical engineering' : 'bachelor of science in electrical engineering',
 u'"Masters in Compter Science"' : 'master of science in computer science',
 u'u"decision scientist' : 'business analyst',
 u'Bachelor of Engineering in Engineering' : 'bachelor of science in electrical engineering',
 u'High School' : '',
 u'Bachelors of Science in Hman Resorce Management' :'Bachelors in Administration',
 u'director of engineering & facilities' : 'director of engineering',
 u'human resource director' : 'recuiter',
 u'architect / lead systems engineer' : 'software engineer',
 u'BS in Bsiness Administration' : 'Bachelors in Administration',
 u'u"customer system engineer / advocate' : 'software engineer',
 u'"Bachelors in Bsiness Management"': 'Bachelors in Administration',
 'master of science in architectre in architectre' : 'master of science in architecture',
 'data analyst & operation assistant' : 'data analyst',
 'data analyst & wholesale manager assistant' : 'data analyst',
  'bachelor of science in architectre' : 'bachelor of science in architecture'
}

title_map2 = {
    "process engineering (technical services)": "process engineer",
    "b.s. in mechanical engineering in with a focs in mechatronics": "bachelor of science in mechanical engineering",
    "aerospace engineer (contract)": "aerospace engineer",
    "lead electromechanical engineer": "senior electromechanical engineer",
    "senior engineering manager": "senior engineer manager",
    "lead engineer & founder": "founder",
    "masters in chemical engineering": "master of science in chemical engineering",
    "manager/assistant director": "manager",
    "associates of science in general science": "diploma in general science",
    "bachelor of science in engineering in indstrial engineering": "bachelor of science in indstrial engineering",
    "associates of applied science in compter aided drafting and design": "diploma in cad",
    "engineering technician iii": "technician",
    "associate of science degree in mechanical engineering": "diploma in mechanical engineering",
    "engineering technician": "technician",
    "engineering project manager - stores": "engineering project manager",
    "bs in civil engineering": "bachelor of science in civil engineering",
    "preparation and evalation of projects": "engineering project manager",
    "assistant and supervisor of engineering projects": "engineering project manager",
    "in graphic design": "bachelor of science in graphic design",
    "carpenter restorer in restoration of works of art": "carpenter",
    "bs in chemical engineering": "bachelor of science in chemical engineering",
    "b.s. in physics in physics": "bachelor of science in physics",
    "b.s. in mechanical engineering": "bachelor of science in mechanical engineering",
    "senior plant engineer": "senior engineer",
    "project manager - corporate engineering": "project manager",
    "mba in exective": "master in business administration",
    "associate (did not complete) in civil engineering": "diploma in civil engineering",
    "senior structural engineering technician": "senior technician",
    "bachelor of science in civil engineering in fndamentals of engineering exam": "bachelor of science in civil engineering", 
    "as in mechanical & general engineering technology": "bachelor of science in mechanical engineering",
    "bs in manfactring technology": "bachelor of science in manufacturing technology",
    "ms in organizational leadership and qality": "master of science in organizational leadership and qality",
    "master in compters in compters": "master of science in computer science",
    "sr. engineering manager": "senior engineering manager",
    "engineering / quality manager": "quality manager",
    "bachelor of science in compter science": "bachelor of science in computer science",
    "associates degree in compter aided design in atodesk": "diploma in cad",
    "a.a. in interdisciplinary stdies in interdisciplinary stdies": "diploma in interdisciplinary studies",
    "finance director/sales manager": "finance director",
    "associates in stationary engineering": "diploma in stationary engineering",
    "in teaching": "bachelor in teaching",
    "in acconting and finance": "bachelor of science in finance",
    "finance manager/director":"finance director",
    "bachelor of economics in economics": "bachelor in economics",
    "mba": "master in business administration",
    "master of bsiness administration in capital markets and asset management immersion": "master in business administration",
    "bachelor of science in acconting": "bachelor of science in accounting",
    "b.a. in bsiness administration": "bachelor in business administration",
    "finance director/secondary director":"finance director",
    "finance director /sales manager":"finance director",
    "m.a. in conseling": "master in counseling",
    "in accounting": "bachelor of science in accounting",
    "in finance": "bachelor of science in finance",
    "associates degree/bachelors in criminal science/jstice":"bachelors in criminal science",
    "ba in bsiness administration":"bachelor in business administration",
    "bs in finance": "bachelor of science in finance",
    "master of bsiness administration in strategy and finance": "master in business administration",
    "bachelor in management/finance": "bachelor of science in finance",
    "in finance": "bachelor of science in finance",
    "in bsiness economics": "bachelor in economics",
    "bs and gradate work": "bachelor of science in finance",
    "usn in usn.": "bachelor of science in finance",
    "sales manager/finance director": "finance director",
    "finance director/secondary director": "finance director",
    "bachelor of liberal arts and science in liberal arts and science": "bachelor of science in liberal arts",
    "senior financial analyst/office manager":"senior financial analyst",
    "richard m squire and associates llc - jenkintown":"associate",
    "libereal arts in compter science": "bachelor of science in liberal arts",
    "masters in psychology and sports management/marketing": "master in psychology",
    "in conseling edcation": "bachelor in counseling education",
    "director of finance": "finance director",
    "bs in mechanical engineering": "bachelor of science in mechanical engineering",
    "bachelor of science in bsiness administration in bsiness administration": "bachelor in business administration", 
    "bs in acconting": "bachelor of science in accounting",
    "masters in bsiness administration": "master in business administration",
    "masters in bsiness administration": "master in business administration",
    "certification in bsiness essentials":"diploma in business",
    "business/financial analyst": "business analyst",
    "bachelor of science in finance in finance": "bachelor of science in finance",
    "ba in commnications": "bachelor in communications",
    "bachelor of bsiness administration in corporate finance": "bachelor of science in finance",
    "bs in economics in international economics and trade":"bachelor in economics",
    "masters in economics in international trade": "master in economics",
    "masters in acconting": "master of science in accounting",
    "bachelor of bsiness administration in finance":"bachelor of science in finance",
    "b.s.": "bachelor of science in finance",
    "m.b.a.": "master in business administration",
    "bachelor of science in bsine ss":"bachelor in business administration",
    "bachelors of science in corporate finance/ bsiness management": "bachelor of science in finance",
    "master of science in finance in information technology and electronic commerce": "master of science in finance",
    "in finance and real estate":"bachelor of science in finance",
    "financial analyst ii": "senior financial analyst",
    "bachelors in finance": "bachelor of science in finance",
    "master of bsiness administration": "master in business administration",
    "bachelor of arts in bsiness administration":"bachelor in business administration",
    "masters in finance": "master of science in finance",
    "bachelor of science in economics in economics":"bachelor in economics",
    "master of science in finance in information technology and electronic commerce": "master of science in finance",   
    "bs in acconting":"bachelor of science in accounting",
    "mba in finance": "master in business administration",
    "bachelors in bsiness administration finance":"bachelor in business administration",
    "associates of science degree":"diploma in science",
    "bachelors in bsiness administration in finance in bsiness administration":"bachelor in business administration",
    "bachelor of bsiness administration in finance":"bachelor in business administration",
    "bs in acconting":"bachelor of science in accounting",
    "bachelors of science in hman resorce management":"bachelor in human resource management",
    "recuiter":"recruiter",
    "bachelor of science in bsiness administration/personnel management":"bachelor in business administration",
    "director of facilities management/hr":"hr director",
    "bachelor of science in bsiness management":"bachelor of science in management",
    "masters in hman resorces management": "master in human resource management",
    "human resource associate (contract)":"human resource associate",
    "master of pblic administration in pblic administration":"master in public administration",
    "in hman resorce management":"master in human resource management",
    "director of human resources":"hr director",
    "bba":"bachelor in business administration",
    "in bank":"human resource associate",
    "bachelors of arts in hman resorces":"bachelor in human resource management",
    "hr recruiter":"recruiter",
    "human resource recruiter - client":"recruiter",
    "bachelor of science degree in bsiness management":"bachelor of science in management",
    "masters degree in core":"master in human resource management",
    "bba in bsiness administration":"bachelor in business administration",
    "ma in organizational management specialization hman resorces":"master in human resource management",
    "director of human capital":"hr director",
    "instructor/recruiter":"recruiter",
    "driver recruiter":"recruiter",
    "corporate recruiter":"recruiter",
    "associates of art bsiness in art bsiness":"diploma in business",
    "interim human resources director":"hr director",
    "bachelor of arts in organizational development":"bachelor of science in organizational leadership",
    "masters of science in organizational leadership":"master of science in organizational leadership",
    "bachelors in bsiness administration":"bachelor in business administration",
    "director of human resources":"hr director",
    "masters in hman resorces development":"master in human resource management",
    "in bsiness administration and management":"bachelor in business administration",
    "mba in bsiness administration":"master in business administration",
    "n master of bsiness administration":"master in business administration",
    "bs in bsiness management":"bachelor of science in management",
    "in hman resorce certification":"diploma in human resource",
    "in general stdies":"bachelor in general studies",
    "director of talent management":"hr director",
    "director human resources":"hr director",
    "a.a.s. in bsiness":"diploma in business",
    "ba in sociology":"bachelor of arts in sociology",
    "raskin & associates":"associate",
    "associate of science in bsiness":"diploma in business",
    "manager/marketing director":"marketing director",
    "activities director / marketing director":"marketing director",
    "community admissions & marketing director":"marketing director",
    "activities director/ recovery coach / marketing":"marketing director",
    "associates of science in marketing":"diploma in marketing",
    "bachelors of science in marketing":"bachelor of science in marketing",
    "bachelor of science in bsiness and economics":"bachelor in economics",
    "bachelor of arts in commnication stdies":"bachelor in communications",
    "bachelor of science in bsiness administration":"bachelor in business administration",
    "bachelor of arts in american stdies":"bachelor of arts in american studies",
    "director of marketing": "marketing director",
    "marketing director/consultant":"marketing director",
    "in english literatre":"bachelor of arts in english literatre",
    "bachelors of science":"bachelor of science",
    "bs in marketing":"bachelor of science in marketing",
    "master of pblic administration in m health information systems":"master in public administration",
    "bachelor of arts in marketing management & pblic relations":"bachelor of science in marketing",
    "bba in marketing":"bachelor of science in marketing",
    "in sales":"bachelor in sales",
    "masters in bsiness administration":"master in business administration",  
    "associate of arts in telecommnications in telecommnications":"diploma in telecommunications",
    "bachelor of arts in mass commnications in pblic relation":"bachelor in communications",
    "masters of science in hman services administration in overall gradate":"master in human resource management",
    "marketing director/administrator":"marketing director",
    "bachelor of science in c technology":"bachelor of science in industrial engineering"
} 

In [15]:
for i in np.arange(len(df_keep)):
    for j in np.arange(len(df_keep.events[i])):
        job_title = df_keep.events[i][j][1]
        if job_title in title_map_ford.keys():
            val = title_map_ford[job_title]
            df_keep.events[i][j] = (x[0], val, x[2])
        elif job_title in title_map.keys():
            val = title_map[job_title]
            df_keep.events[i][j] = (x[0], val, x[2])
        elif job_title in title_map2.keys():
            val = title_map2[job_title]
            df_keep.events[i][j] = (x[0], val, x[2])

In [16]:
df_resumes.head()

,Job_Title,events,userID
0,architect,"[(2010, bachelor of science in architecture, 0...",architect_0
1,architect,"[(2001, high school, 0), (2003, higher seconda...",architect_1
2,architect,"[(2016, junior pega architect, 1), (2020, bach...",architect_2
3,architect,"[(1990, master in arch, 0), (1990, architect, ...",architect_4
4,architect,"[(2008, in architectre, 0), (2008, architect m...",architect_5


In [17]:
df_keep.head()

,Job_Title,events,userID
0,sales_manager,"[(2010, , 0), (2015, sales, 1), (V, current in...",Sales Manager.csv_36
1,sales_manager,"[(2012, hs diploma, 0), (2016, sales, 1)]",Sales Manager.csv_54
2,sales_manager,"[(1999, high school diploma, 0), (2017, sales,...",Sales Manager.csv_87
3,sales_manager,"[(1999, high school diploma, 0), (2017, sales ...",Sales Manager.csv_144
4,sales_manager,"[(1987, bachelor of science, 0), (V, engineer,...",Sales Manager.csv_196


In [18]:
my_titles

['sales_manager',
 'software_architect',
 'software_engineer',
 'vp_of_engineering',
 'vp_of_marketing']

In [20]:
if 1 == 0:
    df_keep.loc[df_keep.Job_Title == my_titles[0], 
            ['userID', 'events', 'Job_Title']].to_json(path_or_buf = '../Data/job_profiles/sales_manager.json',
                                                       orient = 'records', lines = True)
    print('file written!')

In [38]:
titles_keep = get_titles(df_keep)
sorted(titles_keep.items(), key = operator.itemgetter(1), reverse = True)

[('software engineer', 100),
 ('software architect', 42),
 ('senior software engineer', 33),
 ('vp of engineering', 22),
 ('data architect', 20),
 ('vp of marketing', 16),
 ('sales', 16),
 ('master in business administration', 14),
 ('marketing director', 11),
 ('bachelor of science in electrical engineering', 10),
 ('master of science in computer science', 9),
 ('sr software engineer', 9),
 ('bachelor in business administration', 9),
 ('', 6),
 ('senior software architect', 6),
 ('master of science in information technology', 6),
 ('senior architect', 6),
 ('bachelor of science in computer science', 6),
 ('director', 5),
 ('senior engineer', 4),
 ('director of engineering', 4),
 ('software engineer intern', 4),
 ('vp of operations', 4),
 ('bachelor of science in mechanical engineering', 4),
 ('engineer manager', 3),
 ('bachelor of science in engineering', 3),
 ('software development engineer', 3),
 ('data engineer', 3),
 ('high school diploma', 3),
 ('master of science in electrical e

In [70]:
df_keep = df_keep.reset_index(drop = True)
for i in np.arange(len(df_keep)):
    for j in np.arange(len(df_keep.events[i])):
        new = df_keep.events[i][j][1].replace('.', '').replace('compter', 'computer').replace('high school', 'hs').replace('bsiness', 'business')
        
        x_new = (x[0], new, x[2])
        df_keep.events[i][j] = x_new

In [51]:
###### uncomment to see people's events
# print_events(3)

In [17]:
for i in np.arange(len(df_resumes)):
    for j in np.arange(len(df_resumes.events[i])):
        new = df_resumes.events[i][j][1].replace('.', '').replace('masters', 'master').replace('bachelors', 'bachelor')
        new = new.replace('bachelor of science', 'bs').replace('master in arts', 'ma').replace('vice president', 'vp')\
            .replace('master of science', 'ms')
        new = new.replace('master of bsiness administration', 'mba').replace('master in bsiness administration', 'mba')
        new = new.replace('compter', 'computer').replace('bsiness', 'business').replace('architectre', 'architecture')
        #new = re.sub('mba(.*)', 'mba', new)
        x_new = (x[0], new, x[2])
        df_resumes.events[i][j] = x_new

In [67]:
[df_resumes.events]

0    [(2010, bs in architecture, 0), (2010, archite...
1    [(2010, high school, 0), (2010, higher seconda...
2    [(2010, junior pega architect, 0), (2010, bs i...
3    [(2010, master in arch, 0), (2010, architect, ...
4    [(2010, in architectre, 0), (2010, architect m...
Name: events, dtype: object

In [20]:
def get_titles(df):
    titles = dict()
    for i in np.arange(len(df)):
        for j in np.arange(len(df.events[i])):
            job = df.events[i][j][1]
            if job in titles.keys():
                titles[job] += 1
            else:
                titles[job] = 1
    return titles

In [94]:
titles = get_titles(df_resumes)

In [96]:
sorted(titles.items(), key = operator.itemgetter(1), reverse = True)

[('', 1126),
 ('software engineer', 813),
 ('sales', 599),
 ('data analyst', 497),
 ('financial analyst', 466),
 ('business analyst', 430),
 ('architect', 292),
 ('data scientist', 260),
 ('marketing director', 189),
 ('recruiter', 182),
 ('auditor', 145),
 ('high school diploma', 140),
 ('vp of operations', 135),
 ('bs in computer science', 132),
 ('senior software engineer', 127),
 ('night auditor', 126),
 ('senior financial analyst', 105),
 ('engineer manager', 102),
 ('ms in computer science', 97),
 ('director of marketing', 89),
 ('bs in business administration', 83),
 ('data analyst intern', 79),
 ('data engineer', 77),
 ('vp of engineering', 70),
 ('director', 70),
 ('bs in mechanical engineering', 69),
 ('director of operations', 63),
 ('software architect', 62),
 ('vp of marketing', 61),
 ('finance director', 60),
 ('intern architect', 58),
 ('data scientist intern', 56),
 ('project architect', 55),
 ('bs in electrical engineering', 54),
 ('engineering intern', 53),
 ('operati

replace bachelor of science with bs, master of science with ms, vice president with vp

In [29]:
sorted(titles.items(), key = operator.itemgetter(1), reverse = True)

[('software engineer', 813),
 ('', 731),
 ('sales', 599),
 ('data analyst', 497),
 ('financial analyst', 466),
 ('business analyst', 430),
 ('architect', 292),
 ('data scientist', 260),
 ('marketing director', 189),
 ('recruiter', 182),
 ('auditor', 145),
 ('high school diploma', 140),
 ('vp of operations', 135),
 ('senior software engineer', 127),
 ('night auditor', 126),
 ('senior financial analyst', 105),
 ('engineer manager', 102),
 ('director of marketing', 89),
 ('data analyst intern', 79),
 ('data engineer', 77),
 ('director', 70),
 ('bachelor of science in compter science', 65),
 ('director of operations', 63),
 ('software architect', 62),
 ('vp of marketing', 61),
 ('finance director', 60),
 ('intern architect', 58),
 ('master of science in compter science', 57),
 ('vp of engineering', 57),
 ('bachelor of science in bsiness administration', 56),
 ('data scientist intern', 56),
 ('project architect', 55),
 ('mba', 53),
 ('engineering intern', 53),
 ('operation analyst', 52),
 (

### Concatenating JSON files

In [21]:
! ls ../Data/job_profiles/

architect                   marketing_director
auditor                     marketing_director.json
business_analyst            operations_analyst
data_scientist.json         product_manager
engineering_manager         recruiter
engineering_manager.json    sales_manager.json
finance_manager             software_architect.json
finance_manager.json        software_engineer.json
financial_analyst           vp_of_engineering.json
hr_manager                  vp_of_marketing.json
human_resource_manager      vp_of_operation
human_resource_manager.json


In [92]:
engineering_manager = [
    {'userID': 'Engineering Manager.csv_0', 'events': [[2011, 'master of science in chemical engineering', 0], [2011, 'process engineer', 1], [2017, 'phd in environmental engineering', 0]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_1', 'events': [[2015, 'engineer', 1], [2016, 'engineer manager', 1], [2017, 'bachelor of science in mechanical engineering', 0]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_2', 'events': [[2006, 'bachelor of science in aerospace engineering', 0], [2007, 'aerospace engineer', 1], [2008, 'senior electromechanical engineer', 1], [2011, 'senior engineer manager', 1], [2017, 'founder', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_3', 'events': [[2010, 'engineering intern', 1], [2014, 'manager', 1], [2015, 'diploma in general science', 0], [2019, 'bachelor of science in indstrial engineering', 0]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_5', 'events': [[2008, 'quality engineering technician', 1], [2011, 'quality engineering technician', 1], [2012, 'quality engineering team leader', 1], [2013, 'bachelors in indstrial engineering technology', 0], [2013, 'quality engineer', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_7', 'events': [[2012, 'diploma in cad', 0], [2012, 'technician', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_10', 'events': [[1997, 'engineer', 1], [2003, 'bachelor of science in computer science', 0], [2005, 'director of software engineering', 1], [2006, 'director of software engineering', 1], [2010, 'director of software engineering', 1], [2013, 'manager of software engineering', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_11', 'events': [[2006, 'engineering intern', 1], [2007, 'engineering intern', 1], [2008, 'engineering intern', 1], [2009, 'bachelor of science in chemical engineering', 0], [2011, 'associate engineer', 1], [2012, 'senior associate engineer', 1], [2015, 'engineer', 1], [2016, 'engineer', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_15', 'events': [[2015, 'diploma in mechanical engineering', 0], [2016, 'engineering intern', 1], [2017, 'engineering intern', 1], [2018, 'bachelor of science in mechanical engineering', 0]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_16', 'events': [[2015, 'technician', 1], [2016, 'diploma in cad', 0]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_18', 'events': [[1996, 'engineering services', 1], [2012, '', 0], [2014, '', 0]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_19', 'events': [[2006, 'b.s. in civil engineering', 0], [2007, 'project engineer', 1], [2013, 'engineering project manager', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_20', 'events': [[2008, 'preparation and evalation of projects.', 0], [2008, 'supervisor of agricultural engineering projects', 1], [2009, 'corse certificate in formlation', 0], [2009, 'engineering project manager', 1], [2010, 'corse certificate in landscape design of gardens.', 0], [2011, 'corse certificate in nero-lingistic sales programming (nlsp).', 0], [2013, 'corse certificate in qality management systems and training of internal aditors (iso 9001).', 0]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_22', 'events': [[1983, '', 0], [1984, 'certificate in electrical systems design', 0], [1991, 'engineering designer', 1], [1996, 'supervisor of engineering', 1], [2011, 'engineering project manager', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_23', 'events': [[1998, 'bachelor of science in civil engineering', 0], [2000, 'sales engineer', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_25', 'events': [[2000, '', 0], [2001, 'engineer manager', 1], [2010, 'bachelor of science in graphic design', 0]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_26', 'events': [[2010, '', 0], [2011, 'engineering intern', 1], [2014, 'engineering intern', 1], [2015, 'bachelor of science in engineering science', 0]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_27', 'events': [[2005, 'master of science in computer science', 0], [2014, 'senior engineering manager', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_28', 'events': [[1983, 'bachelor of science in mechanical engineering', 0], [1985, 'bachelor of science in manufacturing technology', 0], [1998, 'industrial engineering manager', 1], [2002, 'master of science in organizational leadership and qality', 0], [2007, 'quality manager', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_29', 'events': [[2014, 'engineering intern', 1], [2017, 'engineering intern', 1], [2018, 'bachelor of science in civil engineering in fndamentals of engineering exam', 0]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_30', 'events': [[2007, 'engineering technician intern', 1], [2008, 'diploma in civil engineering', 0], [2008, 'senior technician', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_31', 'events': [[1982, 'bachelor of science in mechanical engineering', 0], [1982, 'senior engineer', 1], [1986, 'project manager', 1], [1998, 'master in business administration', 0], [2009, 'director of engineering', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_34', 'events': [[2017, 'engineering intern', 1], [2018, 'bachelor of science in physics', 0]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_35', 'events': [[1996, 'bachelor of science in chemical engineering', 0], [1996, 'engineer manager', 1]], 'Job_Title': 'Engineering Manager.csv'}
    , {'userID': 'Engineering Manager.csv_36', 'events': [[2000, 'carpenter', 0], [2017, 'engineer manager', 1]], 'Job_Title': 'Engineering Manager.csv'}
]
#pd.DataFrame(engineering_manager)

In [30]:
results = []
with open('../Data/job_profiles/vp_of_engineering.json') as f:
  for line in f:
      line = line.strip()
      #line = ast.literal_eval(line)
      results.append(line)

In [68]:
vpe_df = pd.read_json('../Data/job_profiles/vp_of_engineering.json', lines = True)
sm_df = pd.read_json('../Data/job_profiles/sales_manager.json', lines = True)
sa_df = pd.read_json('../Data/job_profiles/software_architect.json', lines = True)
se_df = pd.read_json('../Data/job_profiles/software_engineer.json', lines = True)
vpm_df = pd.read_json('../Data/job_profiles/vp_of_marketing.json', lines = True)
ds_df = pd.read_json('../Data/job_profiles/data_scientist.json')

In [69]:
r2 = pd.concat([vpe_df, sm_df, sa_df, se_df, vpm_df, ds_df])

In [73]:
! ls ../Data/job_profiles/

architect                   marketing_director
auditor                     marketing_director.json
business_analyst            operations_analyst
data_scientist.json         product_manager
engineering_manager         recruiter
engineering_manager.json    sales_manager.json
finance_manager             software_architect.json
finance_manager.json        software_engineer.json
financial_analyst           vp_of_engineering.json
hr_manager                  vp_of_marketing.json
human_resource_manager      vp_of_operation
human_resource_manager.json


In [29]:
res_df

,Job_Title,events,userID
0,data_scientist,"[[2012, bachelor of science in mathematics, 0]...",data_scientist_5
1,data_scientist,"[[2009, bachelor of science in biomedical engi...",data_scientist_6
2,data_scientist,"[[2016, master in compter engineering in compt...",data_scientist_10
3,data_scientist,"[[2010, bachelor of science in chemistry, 0], ...",data_scientist_17
4,data_scientist,"[[2008, bachelor of science in civil engineeri...",data_scientist_20
5,data_scientist,"[[2010, bachelor in vehicle engineering, 0], [...",data_scientist_21
6,data_scientist,"[[2005, bachelor of arts in business administr...",data_scientist_27
7,data_scientist,"[[1990, bachelor of science in electrical engi...",data_scientist_38
8,data_scientist,"[[2010, bachelor of compter science in tools a...",data_scientist_52
9,data_scientist,"[[2013, bachelor of arts in applied mathematic...",data_scientist_55


In [28]:
pd.DataFrame(res_list).iloc[:, 0].str.split("', '", expand = True).head()

,0,1
0,{'userID': 'architect_0,"events': [(2010, 'bachelor of science in archi..."
1,{'userID': 'architect_4,"events': [(1990, 'master in architecture', 0),..."
2,{'userID': 'architect_7,"events': [(1996, 'bachelor of science in archi..."
3,{'userID': 'architect_9,"events': [(2014, 'bachelor of science in archi..."
4,{'userID': 'architect_12,"events': [(2016, 'architect intern', 1), (2017..."


In [65]:
job_list = ['human_resource_manager.json', 'engineering_manager', 'finance_manager', 'financial_analyst', 'marketing_director', 
            'operations_analyst', 'product_manager', 'recruiter', 'vp_of_operation', 'architect', 'auditor', 'business_analyst']

# missing: sales_associate, operation manager, 

results = []
for j in job_list:
    with open('../Data/job_profiles/' + j) as f:
      for line in f:
          line = line.strip()
          line = ast.literal_eval(line)
          results.append(line)

In [71]:
r1 = pd.DataFrame(results)
job_results = pd.concat([r1, r2])

- Alvira - (Engineering Manager, Finance Manager, Financial Analyst, Human Resource Manager, Marketing Director)
- Asmita - (Operation Analyst, Operation Manager, Product Manager, Recruiter, Sales Associate)
- Ford - (Sales Manager, Software Architect, Software Engineer, VP of Engineering, VP of Marketing)
- Nishan - (VP of Operations, architect, auditor, business_analyst)
- Spencer - (business_strategist, data_analyst, data_engineer, data_scientist)

In [75]:
job_results.to_json(path_or_buf='../Data/job_profiles/all_jobs.json', orient = 'records', lines = True)

In [76]:
pd.read_json('../Data/job_profiles/all_jobs.json', lines = True)

,Job_Title,events,userID
0,Human Resource Manager.csv,"[[2000, bachelor in human resource management,...",Human Resource Manager.csv_1
1,Human Resource Manager.csv,"[[1993, in government, 0], [2015, assist the d...",Human Resource Manager.csv_4
2,Human Resource Manager.csv,"[[1995, bachelor in business administration, 0...",Human Resource Manager.csv_5
3,Human Resource Manager.csv,"[[2005, bachelor of science in management, 0],...",Human Resource Manager.csv_6
4,Human Resource Manager.csv,"[[2012, bachelor of science in management, 0],...",Human Resource Manager.csv_19
5,Human Resource Manager.csv,"[[1995, , 0], [2003, master in human resource ...",Human Resource Manager.csv_24
6,Human Resource Manager.csv,"[[1992, bachelor in business administration, 0...",Human Resource Manager.csv_32
7,Human Resource Manager.csv,"[[2005, bachelor in sales, 0], [2007, high sch...",Human Resource Manager.csv_47
8,Human Resource Manager.csv,"[[1997, , 0], [2016, recruiter, 1]]",Human Resource Manager.csv_54
9,Human Resource Manager.csv,"[[1998, human resource associate, 1], [2005, r...",Human Resource Manager.csv_56
